# Polymarket Analyzer - Interactive Exploration

This notebook demonstrates the core capabilities of the Polymarket Analyzer toolkit:

1. **Connect** to prediction market platforms
2. **Scan** for trading opportunities
3. **Analyze** with research-backed strategies
4. **Backtest** strategies with Monte Carlo simulation
5. **Visualize** results and risk metrics

---

## Setup

In [ ]:
import sys
sys.path.insert(0, '..')

import asyncio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

from src.adapters import PolymarketAdapter, KalshiAdapter
from src.strategies import FavoriteLongshotStrategy, SingleConditionArbitrage
from src.strategies.multi_arb import MultiOutcomeArbitrage
from src.core.config import Credentials

# For async in notebooks
import nest_asyncio
nest_asyncio.apply()

print(f"Notebook initialized: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---
## 1. Connect to Polymarket

Fetch live market data from Polymarket's CLOB (Central Limit Order Book).

In [ ]:
async def fetch_markets(limit=100):
    """Fetch active markets from Polymarket."""
    adapter = PolymarketAdapter()
    await adapter.connect()
    markets = await adapter.get_markets(active_only=True, limit=limit)
    await adapter.disconnect()
    return markets

# Fetch markets
markets = asyncio.run(fetch_markets(200))
print(f"Fetched {len(markets)} active markets")

# Convert to DataFrame for analysis
df = pd.DataFrame([
    {
        'id': m.id,
        'question': m.question[:60] if m.question else 'Unknown',
        'yes_price': m.yes_price,
        'no_price': m.no_price,
        'volume': m.volume or 0,
        'liquidity': m.liquidity or 0,
        'high_prob': max(m.yes_price, m.no_price),
        'sum_prices': m.yes_price + m.no_price,
    }
    for m in markets
])

df.head(10)

### Market Statistics

In [ ]:
print("=" * 50)
print("MARKET STATISTICS")
print("=" * 50)
print(f"Total Markets: {len(df)}")
print(f"Total Volume: ${df['volume'].sum():,.0f}")
print(f"Total Liquidity: ${df['liquidity'].sum():,.0f}")
print(f"\nHigh-Probability Markets (>90%): {len(df[df['high_prob'] >= 0.90])}")
print(f"Extreme Favorites (>95%): {len(df[df['high_prob'] >= 0.95])}")
print(f"\nArbitrage Candidates (sum != 1.00): {len(df[abs(df['sum_prices'] - 1.0) > 0.003])}")

---
## 2. Strategy: Favorite-Longshot Bias

The favorite-longshot bias is a well-documented behavioral phenomenon:
- **Long shots** (low probability) are systematically **overpriced**
- **Favorites** (high probability) are systematically **underpriced**

**Research**: Prospect Theory (Kahneman & Tversky, 1979), NBER Working Paper 15923

In [ ]:
# Initialize strategy
fl_strategy = FavoriteLongshotStrategy(
    min_probability=0.90,  # Only look at >90% favorites
    min_edge=0.01          # Require at least 1% edge
)

# Scan for opportunities
fl_opportunities = []
for market in markets:
    opp = fl_strategy.check_market(market)
    if opp:
        fl_opportunities.append({
            'question': market.question[:50] if market.question else 'Unknown',
            'side': opp.side,
            'price': opp.current_price,
            'fair_value': opp.estimated_fair_value,
            'edge': opp.edge,
            'confidence': opp.confidence,
            'volume': market.volume or 0,
        })

fl_df = pd.DataFrame(fl_opportunities)
fl_df = fl_df.sort_values('edge', ascending=False)

print(f"\nFavorite-Longshot Opportunities: {len(fl_df)}")
print(f"Average Edge: {fl_df['edge'].mean():.2%}")
print(f"Max Edge: {fl_df['edge'].max():.2%}")

fl_df.head(10)

### Visualize Favorite-Longshot Opportunities

In [ ]:
if len(fl_df) > 0:
    fig = px.scatter(
        fl_df,
        x='price',
        y='edge',
        size='volume',
        color='confidence',
        hover_name='question',
        title='Favorite-Longshot Opportunities: Edge vs Price',
        labels={'price': 'Market Price', 'edge': 'Estimated Edge', 'confidence': 'Confidence'},
        color_continuous_scale='RdYlGn',
    )
    fig.add_hline(y=0.02, line_dash="dash", line_color="green", annotation_text="2% Edge Threshold")
    fig.show()
else:
    print("No opportunities found - market may be efficient right now")

---
## 3. Strategy: Single-Condition Arbitrage

When YES + NO ≠ $1.00, risk-free profit exists:
- **Sum < $1.00**: Buy both YES and NO, guaranteed profit
- **Sum > $1.00**: Sell both (if possible), guaranteed profit

**Research**: arXiv:2508.03474 documented $28.3M extracted via this method

In [ ]:
# Initialize strategy
arb_strategy = SingleConditionArbitrage(min_profit_pct=0.003)

# Scan for opportunities
arb_opportunities = []
for market in markets:
    opp = arb_strategy.check_market(market)
    if opp:
        arb_opportunities.append({
            'question': market.question[:50] if market.question else 'Unknown',
            'yes_price': opp.yes_price,
            'no_price': opp.no_price,
            'sum_prices': opp.sum_prices,
            'profit_pct': opp.profit_pct,
            'action': opp.action,
        })

arb_df = pd.DataFrame(arb_opportunities)
if len(arb_df) > 0:
    arb_df = arb_df.sort_values('profit_pct', ascending=False)

print(f"\nArbitrage Opportunities: {len(arb_df)}")
if len(arb_df) > 0:
    print(f"Average Profit: {arb_df['profit_pct'].mean():.2%}")
    print(f"Max Profit: {arb_df['profit_pct'].max():.2%}")
    display(arb_df.head(10))
else:
    print("No arbitrage opportunities found - markets are efficient")

---
## 4. Monte Carlo Backtest

Simulate the favorite-longshot strategy across many scenarios to estimate:
- Return distribution
- Win rate
- Risk metrics (Sharpe, max drawdown)

In [ ]:
def run_backtest(markets_df, num_simulations=100, capital=1000, bet_size=25, edge=0.02):
    """Run Monte Carlo simulation of favorite-longshot strategy."""
    high_prob_markets = markets_df[markets_df['high_prob'] >= 0.90].copy()
    
    if len(high_prob_markets) == 0:
        return None
    
    returns = []
    win_rates = []
    
    for _ in range(num_simulations):
        balance = capital
        wins = 0
        trades = 0
        
        for _, row in high_prob_markets.iterrows():
            if balance < bet_size:
                break
            
            prob = row['high_prob']
            true_prob = min(0.99, prob + edge)  # Bias adjustment
            
            # Simulate outcome
            won = np.random.random() < true_prob
            if won:
                profit = bet_size * (1 - prob) / prob
                wins += 1
            else:
                profit = -bet_size
            
            balance += profit
            trades += 1
        
        total_return = (balance - capital) / capital
        win_rate = wins / trades if trades > 0 else 0
        
        returns.append(total_return)
        win_rates.append(win_rate)
    
    return {
        'returns': np.array(returns),
        'win_rates': np.array(win_rates),
        'num_trades': len(high_prob_markets),
    }

# Run backtest
results = run_backtest(df, num_simulations=100)

if results:
    print("=" * 50)
    print("BACKTEST RESULTS (100 Simulations)")
    print("=" * 50)
    print(f"Markets Tested: {results['num_trades']}")
    print(f"\nReturn Statistics:")
    print(f"  Mean: {results['returns'].mean():.2%}")
    print(f"  Median: {np.median(results['returns']):.2%}")
    print(f"  Std Dev: {results['returns'].std():.2%}")
    print(f"  5th Percentile: {np.percentile(results['returns'], 5):.2%}")
    print(f"  95th Percentile: {np.percentile(results['returns'], 95):.2%}")
    print(f"\nRisk Metrics:")
    print(f"  Prob. Profitable: {(results['returns'] > 0).mean():.1%}")
    print(f"  Avg Win Rate: {results['win_rates'].mean():.1%}")
    sharpe = results['returns'].mean() / results['returns'].std() if results['returns'].std() > 0 else 0
    print(f"  Sharpe Ratio: {sharpe:.2f}")

In [ ]:
if results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Return distribution
    ax1 = axes[0]
    ax1.hist(results['returns'], bins=20, edgecolor='white', color='steelblue', alpha=0.7)
    ax1.axvline(results['returns'].mean(), color='red', linestyle='--', label=f"Mean: {results['returns'].mean():.1%}")
    ax1.axvline(np.median(results['returns']), color='green', linestyle='--', label=f"Median: {np.median(results['returns']):.1%}")
    ax1.axvline(0, color='black', linewidth=0.5)
    ax1.set_xlabel('Return')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Backtest Return Distribution', fontweight='bold')
    ax1.legend()
    
    # Win rate distribution
    ax2 = axes[1]
    ax2.hist(results['win_rates'], bins=20, edgecolor='white', color='seagreen', alpha=0.7)
    ax2.axvline(results['win_rates'].mean(), color='red', linestyle='--', label=f"Mean: {results['win_rates'].mean():.1%}")
    ax2.set_xlabel('Win Rate')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Win Rate Distribution', fontweight='bold')
    ax2.legend()
    
    plt.tight_layout()
    plt.savefig('../results/backtest_results.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\nSaved: results/backtest_results.png")

---
## 5. Expected Value Analysis

Calculate E(X) under different edge assumptions to understand profitability thresholds.

In [ ]:
def calculate_ev(prob, edge, bet_size=25):
    """Calculate expected value for a bet."""
    true_prob = min(0.99, prob + edge)
    win_profit = bet_size * (1 - prob) / prob
    lose_profit = -bet_size
    return true_prob * win_profit + (1 - true_prob) * lose_profit

# Test different edge scenarios
edge_scenarios = [0.00, 0.01, 0.02, 0.03, 0.04]
probabilities = [0.90, 0.92, 0.94, 0.95, 0.96, 0.97, 0.98]

ev_data = []
for prob in probabilities:
    row = {'Probability': f"{prob:.0%}"}
    for edge in edge_scenarios:
        ev = calculate_ev(prob, edge)
        row[f'{edge:.0%} Edge'] = f"${ev:.2f}"
    ev_data.append(row)

ev_df = pd.DataFrame(ev_data)
print("Expected Value per $25 Bet by Edge Assumption")
print("=" * 60)
ev_df

In [ ]:
# Visualize E(X) surface
fig = go.Figure()

for edge in edge_scenarios:
    evs = [calculate_ev(p, edge) for p in np.linspace(0.85, 0.99, 50)]
    fig.add_trace(go.Scatter(
        x=np.linspace(0.85, 0.99, 50) * 100,
        y=evs,
        name=f'{edge:.0%} Edge',
        mode='lines',
    ))

fig.add_hline(y=0, line_dash="dash", line_color="black")
fig.update_layout(
    title='Expected Value by Probability and Edge Assumption',
    xaxis_title='Market Probability (%)',
    yaxis_title='E(X) per $25 Bet ($)',
    legend_title='Edge',
)
fig.show()

---
## 6. Portfolio Summary

Combine all strategies into a portfolio view.

In [ ]:
print("=" * 60)
print("PORTFOLIO SUMMARY")
print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

print(f"\n📊 Markets Analyzed: {len(df)}")
print(f"💰 Total Volume: ${df['volume'].sum():,.0f}")
print(f"💧 Total Liquidity: ${df['liquidity'].sum():,.0f}")

print(f"\n🎯 FAVORITE-LONGSHOT STRATEGY")
print(f"   Opportunities: {len(fl_df)}")
if len(fl_df) > 0:
    print(f"   Average Edge: {fl_df['edge'].mean():.2%}")
    print(f"   Best Edge: {fl_df['edge'].max():.2%}")

print(f"\n⚖️ ARBITRAGE STRATEGY")
print(f"   Opportunities: {len(arb_df)}")
if len(arb_df) > 0:
    print(f"   Average Profit: {arb_df['profit_pct'].mean():.2%}")

if results:
    print(f"\n📈 BACKTEST METRICS")
    print(f"   Prob. Profitable: {(results['returns'] > 0).mean():.1%}")
    print(f"   Avg Win Rate: {results['win_rates'].mean():.1%}")
    print(f"   Median Return: {np.median(results['returns']):.2%}")

---

## Next Steps

From the CLI, you can:

```bash
# Generate detailed reports
python run.py report generate --strategy favorite_longshot
python run.py report pdf --strategy favorite_longshot

# Run more sophisticated analysis
python run.py analyze risk --correlation
python run.py analyze investigate --fetch-spreads

# Generate visualizations
python run.py visualize dashboard
python run.py visualize honest
```